In [7]:
from google.colab import files
uploaded = files.upload()


Saving spam.csv to spam.csv


In [11]:
# Install required libraries
!pip install -q streamlit pyngrok imbalanced-learn

# Code starts here
import pandas as pd
import pickle
import os
import threading
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from imblearn.over_sampling import SMOTE
from pyngrok import ngrok

# Upload dataset if not already uploaded
if not os.path.exists("spam.csv"):
    from google.colab import files
    uploaded = files.upload()

# Load and clean dataset
df = pd.read_csv("spam.csv", encoding="latin-1")[['Category', 'Message']]
df = df.rename(columns={"Category": "label", "Message": "email"})
df['label'] = df['label'].map({'ham': 0, 'spam': 1})
df.dropna(inplace=True)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(df['email'], df['label'], test_size=0.2, random_state=42, stratify=df['label'])

# Vectorize
vectorizer = CountVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Handle class imbalance
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train_vec, y_train)

# Train model
model = MultinomialNB()
model.fit(X_train_bal, y_train_bal)

# Save model and vectorizer
pickle.dump(model, open("model.pkl", "wb"))
pickle.dump(vectorizer, open("vectorizer.pkl", "wb"))

# Create Streamlit app
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import pickle

model = pickle.load(open("model.pkl", "rb"))
vectorizer = pickle.load(open("vectorizer.pkl", "rb"))

st.title("📧 Email Spam Detection")

new_email = st.text_area("Enter the email text:")

if st.button("Classify Email"):
    if new_email:
        new_email_counts = vectorizer.transform([new_email])
        pred_prob = model.predict_proba(new_email_counts)
        if pred_prob.shape[0] > 0 and pred_prob.shape[1] > 1:
            result = "Spam" if pred_prob[0][1] > 0.4 else "Not Spam"
        else:
            result = "Unable to classify. Please check the input email."
        st.write(f"**Prediction:** {result}")
    else:
        st.warning("⚠️ Please enter an email to classify.")
""")

# Enter your ngrok auth token here (from dashboard.ngrok.com)
ngrok.set_auth_token("2xUOEafCK3R7p14GmaB2SpvQCQo_7ZerVzfG8Fg6nnp6N1boY")  # 🔑 Replace this!

# Run Streamlit in background
def run():
    os.system("streamlit run app.py")

thread = threading.Thread(target=run)
thread.start()

# Get public URL
public_url = ngrok.connect(8501)
print(f"🔗 Your app is live here: {public_url}")


🔗 Your app is live here: NgrokTunnel: "https://f05e-34-118-253-97.ngrok-free.app" -> "http://localhost:8501"


In [12]:
# 🚀 Run Streamlit in background
!streamlit run app.py &>/content/logs.txt &


In [13]:
from pyngrok import ngrok

# Start ngrok tunnel to the streamlit port 8501
public_url = ngrok.connect(8501)
print(f"🔗 Your app is live here: {public_url}")


🔗 Your app is live here: NgrokTunnel: "https://a48b-34-118-253-97.ngrok-free.app" -> "http://localhost:8501"
